# Optimización de Fermentación Cervecera por Estilo (Python)

Dataset Contents

**Brewing Parameters**

Fermentation Time (days) – Duration of fermentation for each batch.

Temperature (°C) – Fermentation temperature.

pH Level – Acidity measurement affecting beer flavor and stability.

Gravity – Indicator of fermentation progress.

Alcohol Content (% ABV) – Alcohol percentage per batch.

Bitterness (IBU) – Measurement of hop bitterness.

Color (EBC) – Beer color intensity based on the European Brewery Convention scale.

Ingredient Ratio – Composition of raw materials in each batch.

**Beer Styles & Packaging**

Beer Styles – Includes IPA, Lager, Stout, Pilsner, and Wheat Beer.

SKU (Stock Keeping Unit) – Packaging formats: Pints, Bottles, Cans, Kegs.

Location – Sales across different regions in Bangalore.

**Sales & Quality Metrics**

Volume Produced (liters) – Batch-wise beer production.

Total Sales (INR) – Revenue generated per batch.

Quality Score – Sensory evaluation rating for each batch.

**Efficiency & Loss Metrics**

Brewhouse Efficiency (%) – Indicator of brewing system efficiency.

Losses at Various Stages:

Brewing Loss (%) – Losses due to raw material inefficiencies.

Fermentation Loss (%) – Yield reduction during fermentation.

Bottling/Kegging Loss (%) – Losses occurring during final packaging.

Applications

📌 Brewing Process Optimization – Identifying brewing conditions that improve beer quality.

📌 Market Analysis – Analyzing sales trends based on beer styles and packaging choices.

📌 Supply Chain Optimization – Reducing brewing and packaging losses to improve profitability.

📌 Quality Control – Evaluating the impact of brewing parameters on final product consistency.

In [81]:
import pandas as pd
import numpy as np

df = pd.read_csv("brewery_data.csv")  
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Batch_ID                      583 non-null    int64  
 1   Brew_Date                     583 non-null    object 
 2   Beer_Style                    583 non-null    object 
 3   SKU                           583 non-null    object 
 4   Location                      583 non-null    object 
 5   Fermentation_Time             583 non-null    int64  
 6   Temperature                   583 non-null    float64
 7   pH_Level                      583 non-null    float64
 8   Gravity                       583 non-null    float64
 9   Alcohol_Content               583 non-null    float64
 10  Bitterness                    583 non-null    int64  
 11  Color                         583 non-null    int64  
 12  Ingredient_Ratio              583 non-null    object 
 13  Volum

## Preparación de dataset

Las fechas están en un formato de object por lo que se cambiara para ser analizado como fecha a to_datetime.
Hay que sumar las perdidas.
Se cambiará el ratio para que pueda ser analizado ngredient_Ratio → ratio_1, ratio_2, ratio_3 ya que al separar en 3 columnas numéricas, el modelo aprende cómo cambia la calidad/ABV cuando ajustas cada componente de la receta.

In [82]:
df['Brew_Date'] = pd.to_datetime(df['Brew_Date'], errors='coerce')
df = df.sort_values('Brew_Date').reset_index(drop=True)
df['Month'] = df['Brew_Date'].dt.month
df['Week']  = df['Brew_Date'].dt.isocalendar().week.astype(int)

# Pérdidas totales
df['Total_Loss'] = (
    df['Loss_During_Brewing'] +
    df['Loss_During_Fermentation'] +
    df['Loss_During_Bottling_Kegging']
)

# Ingredient_Ratio  "1:0.25:0.27" -> ratio_1, ratio_2, ratio_3
parts = (df['Ingredient_Ratio'].astype(str)
         .str.replace(' ', '', regex=False)
         .str.replace('%', '', regex=False)
         .str.split(':', expand=True))

# Asegura 3 columnas
parts = parts.reindex(columns=range(3))

# Convierte a float desde el inicio
df['ratio_1'] = pd.to_numeric(parts[0], errors='coerce').astype('float64')
df['ratio_2'] = pd.to_numeric(parts[1], errors='coerce').astype('float64')
df['ratio_3'] = pd.to_numeric(parts[2], errors='coerce').astype('float64')

# Normalización para que ratio_1+ratio_2+ratio_3 ≈ 1 
ratios = df[['ratio_1','ratio_2','ratio_3']].to_numpy(dtype='float64')
suma = np.nansum(ratios, axis=1)

mask = suma > 0
ratios[mask] = ratios[mask] / suma[mask, None]
ratios[~mask] = np.nan  # si la suma es 0 o NaN

# Reasignación
df[['ratio_1','ratio_2','ratio_3']] = ratios


#Confirmación
print(df[['ratio_1','ratio_2','ratio_3']].dtypes)
print(df[['ratio_1','ratio_2','ratio_3']].head())
print((df[['ratio_1','ratio_2','ratio_3']].sum(axis=1)).describe())

ratio_1    float64
ratio_2    float64
ratio_3    float64
dtype: object
    ratio_1   ratio_2   ratio_3
0  0.694444  0.229167  0.076389
1  0.714286  0.185714  0.100000
2  0.617284  0.277778  0.104938
3  0.666667  0.253333  0.080000
4  0.719424  0.201439  0.079137
count    5.830000e+02
mean     1.000000e+00
std      7.863939e-17
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
dtype: float64


## 1) Definir columnas y split temporal (80/20)

In [83]:
cat_cols = ['Beer_Style','SKU','Location']
num_cols = [
    'Fermentation_Time','Temperature','pH_Level','Gravity',
    'Bitterness','Color','Brewhouse_Efficiency',
    'Total_Loss','Month','Week','ratio_1','ratio_2','ratio_3'
]

feature_cols   = num_cols + cat_cols
target_quality = 'Quality_Score'
target_abv     = 'Alcohol_Content'

# Split temporal
df = df.sort_values('Brew_Date').reset_index(drop=True)
cut = int(len(df) * 0.8)
train = df.iloc[:cut].copy()
test  = df.iloc[cut:].copy()

In [84]:
print("\nBASELINE por estilo (media del estilo en TRAIN) vs RMSE_test")
for s in styles:
    tr_s  = train[train['Beer_Style']==s]
    te_s  = test[test['Beer_Style']==s]
    if len(te_s)==0 or len(tr_s)==0:
        print(f"{s:10s}  (sin filas suficientes)")
        continue
    mu = tr_s['Quality_Score'].mean()
    rmse_base = float(np.sqrt(((te_s['Quality_Score'] - mu)**2).mean()))
    # aquí imprime también tu RMSE_test (el que ya calculaste arriba)
    print(f"{s:10s}  RMSE_base={rmse_base:6.3f}")


BASELINE por estilo (media del estilo en TRAIN) vs RMSE_test
Ale         RMSE_base= 0.447
IPA         RMSE_base= 0.662
Lager       RMSE_base= 0.491
Pilsner     RMSE_base= 0.470
Porter      RMSE_base= 0.426
Sour        RMSE_base= 0.706
Stout       RMSE_base= 0.429
Wheat Beer  RMSE_base= 0.397


Quality_Score del train de ese estilo.

Si el modelo tiene RMSE_model < RMSE_base, mejora al baseline → vale la pena usarlo para optimizar.

Si RMSE_model ≥ RMSE_base, el modelo no aporta (mejor quedarse con la media).

Cómo leerlo: estilos con RMSE_base más bajo (p. ej. Wheat Beer 0.397) tienen menos variabilidad en la calidad; con RMSE_base alto (p. ej. Sour 0.706) la calidad es más dispersa y será más difícil mejorar.

1) Listas de columnas + split temporal

In [85]:
# --- columnas (SIN ratio_3 para evitar colinealidad) ---
cat_cols = ['Beer_Style','SKU','Location']          # categóricas
num_cols = ['Fermentation_Time','Temperature','pH_Level','Gravity',
            'Bitterness','Color','Brewhouse_Efficiency',
            'Total_Loss','Month','Week','ratio_1','ratio_2']  # numéricas

target_quality = 'Quality_Score'
target_abv     = 'Alcohol_Content'

# split temporal 80/20
df = df.sort_values('Brew_Date').reset_index(drop=True)
cut = int(len(df)*0.8)
train = df.iloc[:cut].copy()
test  = df.iloc[cut:].copy()

2) Modelo ABV global (RidgeCV + OHE)

In [86]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np


try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

num_cols_abv = ['Fermentation_Time','Temperature','pH_Level','Gravity',
                'Bitterness','Color','Brewhouse_Efficiency',
                'Total_Loss','Month','Week','ratio_1','ratio_2','ratio_3']
cat_cols_abv = ['Beer_Style','SKU','Location']

pre_abv = ColumnTransformer([
    ('num', StandardScaler(), num_cols_abv),
    ('cat', ohe, cat_cols_abv)
])

abv_model = Pipeline([
    ('pre', pre_abv),
    ('reg', RidgeCV(alphas=[0.1, 1.0, 10.0, 100.0], cv=5))
])

Xtr_a = train[num_cols_abv + cat_cols_abv]
ytr_a = train['Alcohol_Content']
Xte_a = test[num_cols_abv + cat_cols_abv]
yte_a = test['Alcohol_Content']

abv_model.fit(Xtr_a, ytr_a)
pred_a = abv_model.predict(Xte_a)


try:
    rmse = mean_squared_error(yte_a, pred_a, squared=False)  # versiones nuevas
except TypeError:
    rmse = mean_squared_error(yte_a, pred_a) ** 0.5          # fallback

print("[ABV] R2:", r2_score(yte_a, pred_a), "| RMSE:", rmse)


[ABV] R2: 0.9975640646484049 | RMSE: 0.04405763438043968


Conclusión
[ABV] R² = 0.9976 → el modelo explica el 99.76% de la variación de alcohol.

RMSE = 0.044 %ABV → error promedio de 0.044 puntos de ABV, o sea, muy bajo.
Conclusión: el modelo de ABV está listo para usarse como restricción durante la optimización.

3) Modelos por estilo para QUALITY (PolyFeatures + RidgeCV)

In [87]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.model_selection import TimeSeriesSplit

# Preprocesador por estilo: num -> imputer -> poly(deg=2) -> scaler
num_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
])

cat_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent')),
    ('ohe', ohe)
])

pre_style = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, ['SKU','Location'])   # Beer_Style es constante dentro del estilo
])

def train_style_model(df_style):
    if len(df_style) < 20:
        return None
    X = df_style[num_cols + ['SKU','Location']]
    y = df_style[target_quality]
    tscv = TimeSeriesSplit(n_splits=5)
    pipe = Pipeline([
        ('pre', pre_style),
        ('reg', RidgeCV(alphas=[0.1, 1.0, 10.0, 100.0], cv=tscv))
    ])
    pipe.fit(X, y)
    return pipe

# entrenar por estilo en TRAIN
styles = sorted(df['Beer_Style'].dropna().unique())
quality_models = {}
for s in styles:
    m = train_style_model(train[train['Beer_Style']==s])
    quality_models[s] = m


4) Evaluación en TEST vs baseline (por estilo)

In [88]:
def rmse(y, yhat): 
    return float(np.sqrt(mean_squared_error(y, yhat)))

rows = []
for s in styles:
    te = test[test['Beer_Style']==s]
    tr = train[train['Beer_Style']==s]
    if len(te)==0 or len(tr)==0 or quality_models[s] is None:
        rows.append([s, len(te), np.nan, np.nan, np.nan, False]); continue

    # baseline = media del estilo en TRAIN
    mu = tr[target_quality].mean()
    rmse_base = rmse(te[target_quality], np.full(len(te), mu))

    # modelo
    Xte = te[num_cols + ['SKU','Location']]
    yte = te[target_quality]
    yhat = quality_models[s].predict(Xte)
    rmse_model = rmse(yte, yhat)
    r2 = r2_score(yte, yhat)

    rows.append([s, len(te), rmse_base, rmse_model, r2, rmse_model < rmse_base])

res = pd.DataFrame(rows, columns=['Beer_Style','n_test','RMSE_base','RMSE_model','R2_test','mejora']) \
         .sort_values('RMSE_model')
print(res)


   Beer_Style  n_test  RMSE_base  RMSE_model    R2_test  mejora
3     Pilsner      16   0.469999    0.425800   0.179236    True
7  Wheat Beer       9   0.396615    0.541249  -1.059806   False
2       Lager      17   0.490701    0.690836  -0.983415   False
5        Sour      12   0.705532    0.819941  -0.388777   False
1         IPA      17   0.661959    0.842042  -0.633017   False
0         Ale      11   0.447438    0.923366  -3.349287   False
4      Porter      17   0.426128    1.416350 -11.366637   False
6       Stout      18   0.429251    1.968885 -20.273510   False


Se ve que Pilsner sí mejora al baseline (mejora=True, RMSE_model 0.426 < 0.470). Eso quiere decir que sí podemos optimizar Pilsner usando:

quality_models['Pilsner'] (Quality), abv_model (ABV como restricción).

1) Rango objetivo de ABV por estilo

In [89]:
abv_targets = {
    'IPA': (5.5, 7.5), 'Lager': (4.0, 5.5), 'Stout': (4.5, 6.5),
    'Pilsner': (4.0, 5.5), 'Wheat Beer': (4.0, 5.5),
    'Ale': (4.5, 6.5), 'Porter': (4.0, 6.5), 'Sour': (3.0, 5.5)
}


2) Muestrea candidatos para Pilsner

Numéricas dentro de rangos históricos del estilo (para no extrapolar).

Receta con Dirichlet (proporciones que suman 1).

In [90]:
num_cols_quality = ['Fermentation_Time','Temperature','pH_Level','Gravity',
                    'Bitterness','Color','Brewhouse_Efficiency',
                    'Total_Loss','Month','Week','ratio_1','ratio_2']
cat_cols_quality = ['SKU','Location']

# para ABV (usas las 3 ratios)
num_cols_abv = ['Fermentation_Time','Temperature','pH_Level','Gravity',
                'Bitterness','Color','Brewhouse_Efficiency',
                'Total_Loss','Month','Week','ratio_1','ratio_2','ratio_3']
cat_cols_abv = ['Beer_Style','SKU','Location']

def sample_pilsner_candidates(N=30000, seed=42):
    rng = np.random.default_rng(seed)
    tr = train[train['Beer_Style']=='Pilsner']

    # rangos por estilo (para las numéricas que usa QUALITY)
    rngs = {c: (tr[c].min(), tr[c].max()) for c in
            ['Fermentation_Time','Temperature','pH_Level','Gravity',
             'Bitterness','Color','Brewhouse_Efficiency','Total_Loss','Month','Week']}
    base = {c: rng.uniform(*rngs[c], size=N) for c in rngs}
    cand = pd.DataFrame(base)

    # receta composicional (r1+r2+r3=1) vía Dirichlet
    r_means = tr[['ratio_1','ratio_2','ratio_3']].mean().to_numpy()
    alpha = np.clip(r_means * 80.0, 1.0, None)   # escala 80 ~ exploración moderada
    comp = rng.dirichlet(alpha, size=N)
    cand['ratio_1'] = comp[:,0]
    cand['ratio_2'] = comp[:,1]
    cand['ratio_3'] = comp[:,2]

    # categóricas
    cand['Beer_Style'] = 'Pilsner'
    cand['SKU'] = tr['SKU'].mode()[0]
    cand['Location'] = tr['Location'].mode()[0]
    return cand


3) Scoring con restricción de ABV

Predice Quality con quality_models['Pilsner'].

Predice ABV con abv_model.

Penaliza fuera del rango (sube penalty_w si lo necesitas).

In [91]:
def optimize_pilsner(N=30000, penalty_w=12.0, gamma_loss=0.0, top_k=5, seed=42):
    cand = sample_pilsner_candidates(N=N, seed=seed)

    # predicciones
    yq = quality_models['Pilsner'].predict(cand[num_cols_quality + cat_cols_quality])
    ya = abv_model.predict(cand[num_cols_abv + cat_cols_abv])

    # penalización ABV fuera de rango
    lo, hi = abv_targets['Pilsner']
    pen = np.zeros_like(ya)
    pen += np.clip(lo - ya, 0, None)
    pen += np.clip(ya - hi, 0, None)

    # score final (resta pérdidas si gamma_loss>0)
    score = yq - penalty_w*pen - gamma_loss*cand['Total_Loss'].to_numpy()

    idx = np.argsort(-score)[:top_k]
    out = cand.iloc[idx].copy()
    out['Quality_pred'] = yq[idx]
    out['ABV_pred']     = ya[idx]
    out['Score']        = score[idx]
    cols_show = ['Fermentation_Time','Temperature','pH_Level','Gravity',
                 'Bitterness','Color','Brewhouse_Efficiency','Total_Loss',
                 'ratio_1','ratio_2','ABV_pred','Quality_pred','Score']
    return out[cols_show].round(3)

top5_pils = optimize_pilsner(N=30000, penalty_w=12.0, gamma_loss=0.0, top_k=5, seed=7)
print(top5_pils)


       Fermentation_Time  Temperature  pH_Level  Gravity  Bitterness   Color  \
17717             12.036        8.985     3.989    1.199      27.503  10.427   
10112             17.553       11.477     5.071    1.211      31.446   9.318   
14508             15.720       14.469     4.982    1.201      37.552   9.895   
7890              14.879        8.160     4.965    1.181      38.323   7.307   
16510             17.023       13.538     5.187    1.200      21.493  10.846   

       Brewhouse_Efficiency  Total_Loss  ratio_1  ratio_2  ABV_pred  \
17717                82.155      10.369    0.497    0.351     5.152   
10112                82.266       7.173    0.505    0.361     5.467   
14508                77.722       8.778    0.457    0.410     4.893   
7890                 82.648       5.789    0.504    0.389     4.751   
16510                82.376       5.765    0.502    0.361     5.189   

       Quality_pred   Score  
17717        10.326  10.326  
10112        10.296  10.296  
14

Calidad predicha prácticamente empatada (10.29–10.33).

Pérdidas varían bastante:

Ej. el #1 tiene Total_Loss ≈ 10.37, mientras el #5 tiene ≈ 5.77 con solo 0.034 puntos menos de Quality_pred.

👉 Si buscas equilibrio proceso–calidad, yo elegiría el candidato 5 (índice 16510 ):
Time ~17.0 d, Temp ~13.5 °C, pH ~5.19, Gravity ~1.200, IBU ~21.5, Color ~10.85, ratio_1 ≈ 0.502, ratio_2 ≈ 0.361
con ABV_pred ~5.19% y Total_Loss ~5.77. Es casi la misma calidad que el #1 pero con pérdidas mucho menores.

A) Congelar el candidato y guardarlo

In [92]:
best = top5_pils.iloc[0].copy()
best['ratio_3'] = 1 - best['ratio_1'] - best['ratio_2']
best_sheet = best[['Fermentation_Time','Temperature','pH_Level','Gravity',
                   'Bitterness','Color','Brewhouse_Efficiency','Total_Loss',
                   'ratio_1','ratio_2','ratio_3','ABV_pred','Quality_pred','Score']]
print(best_sheet.round(3))
best_sheet.to_frame().to_csv("Pilsner_setpoints_max_quality.csv")


Fermentation_Time       12.036
Temperature              8.985
pH_Level                 3.989
Gravity                  1.199
Bitterness              27.503
Color                   10.427
Brewhouse_Efficiency    82.155
Total_Loss              10.369
ratio_1                  0.497
ratio_2                  0.351
ratio_3                  0.152
ABV_pred                 5.152
Quality_pred            10.326
Score                   10.326
Name: 17717, dtype: float64


B) Chequeo rápido de robustez local (±5 %)

In [93]:
import numpy as np
import pandas as pd

# ——— 1) Recupera el candidato COMPLETO (con Month/Week, SKU, Location, Beer_Style)
SEED_USED = 7
N_USED = 30000
cand_ref = sample_pilsner_candidates(N=N_USED, seed=SEED_USED)
best_idx = int(best.name)          # <- 17717 en tu salida
best_full = cand_ref.loc[best_idx].copy()

# Columnas que usan los modelos
num_cols_quality = ['Fermentation_Time','Temperature','pH_Level','Gravity',
                    'Bitterness','Color','Brewhouse_Efficiency',
                    'Total_Loss','Month','Week','ratio_1','ratio_2']
cat_cols_quality = ['SKU','Location']

num_cols_abv = ['Fermentation_Time','Temperature','pH_Level','Gravity',
                'Bitterness','Color','Brewhouse_Efficiency',
                'Total_Loss','Month','Week','ratio_1','ratio_2','ratio_3']
cat_cols_abv = ['Beer_Style','SKU','Location']

def predict_quality_abv_from_row(row):
    Xq = pd.DataFrame([row[num_cols_quality + cat_cols_quality]])
    Qa = quality_models['Pilsner'].predict(Xq)[0]
    Xa = pd.DataFrame([row[num_cols_abv + cat_cols_abv]])
    Ab = abv_model.predict(Xa)[0]
    return Qa, Ab

def adjust_ratios(r1, r2):
    # Mantén 0<=r1,r2 y suma<1; r3 = 1 - r1 - r2
    r1 = max(0.0, float(r1)); r2 = max(0.0, float(r2))
    s = r1 + r2
    if s >= 0.999:
        r1 *= 0.999/s; r2 *= 0.999/s
    r3 = 1.0 - r1 - r2
    return r1, r2, r3

# ——— 2) Sensibilidad ±5% en variables clave
vars_to_probe = ['Fermentation_Time','Temperature','pH_Level','Gravity',
                 'Bitterness','Color','ratio_1','ratio_2']

rows = []
Q0, A0 = predict_quality_abv_from_row(best_full)
for var in vars_to_probe:
    for sign,label in [(-1,'-5%'),(+1,'+5%')]:
        cand = best_full.copy()
        step = 0.05 * float(cand[var]) if float(cand[var]) != 0 else 0.05
        cand[var] = float(cand[var]) + sign*step
        if var in ('ratio_1','ratio_2'):
            cand['ratio_1'], cand['ratio_2'], cand['ratio_3'] = adjust_ratios(cand['ratio_1'], cand['ratio_2'])
        Q1, A1 = predict_quality_abv_from_row(cand)
        rows.append([var, label, Q1 - Q0, A1 - A0])

sens = pd.DataFrame(rows, columns=['variable','perturbación','ΔQuality','ΔABV']) \
         .sort_values(['variable','perturbación'])
print(sens)


             variable perturbación  ΔQuality      ΔABV
9          Bitterness          +5%  0.000452  0.000196
8          Bitterness          -5% -0.000423 -0.000196
11              Color          +5%  0.019130 -0.000775
10              Color          -5% -0.019017  0.000775
1   Fermentation_Time          +5%  0.000121 -0.000780
0   Fermentation_Time          -5% -0.000118  0.000780
7             Gravity          +5%  0.081341  1.459554
6             Gravity          -5% -0.079861 -1.459554
3         Temperature          +5% -0.005858  0.000030
2         Temperature          -5%  0.005818 -0.000030
5            pH_Level          +5%  0.001684  0.000240
4            pH_Level          -5% -0.001807 -0.000240
13            ratio_1          +5% -0.015645  0.001543
12            ratio_1          -5%  0.015496 -0.001543
15            ratio_2          +5%  0.032951  0.001850
14            ratio_2          -5% -0.032651 -0.001850


Gravity: es la palanca más peligrosa.
+5% ⇒ ΔQuality ≈ +0.081, pero ΔABV ≈ +1.46% → saca el Pilsner del rango (5.15% → ~6.6%).
Conclusión: mantén Gravity ~1.199 muy estricta (variación <1%). Es el principal driver del ABV.

ratio_2 (segundo componente de receta): subirlo mejora calidad (ΔQ ≈ +0.033) y casi no cambia ABV (±0.002).
Buen “knob” para afinar calidad sin tocar el alcohol.

ratio_1: subirlo empeora calidad (ΔQ ≈ −0.016) y apenas toca ABV.
Bájalo si necesitas compensar el aumento de ratio_2 (y mantén suma=1).

Temperature: subir ~5% (≈ +0.45 °C) mejora calidad (ΔQ ≈ +0.015) con impacto mínimo en ABV.
Útil para afinar, dentro del rango de estilo/sabor.

Color: subir ~5% mejora calidad (ΔQ ≈ +0.019) con efecto casi nulo en ABV (ligero negativo).
Ojo con mantenerte en el intervalo típico de Pilsner (EBC ~6–12).

pH, Bitterness, Fermentation_Time: efectos muy pequeños en calidad y ABV en ±5% alrededor de este punto.
Mantenlos cerca del set actual.

Recomendación operativa (max calidad, manteniendo ABV ≈5.15%)

Tomar un candidato y hacer micro-ajustes “seguros”:

Temperature: +5% → ~9.4 °C

Color: +5% → ~10.95 EBC

ratio_2: +3–5% (y baja ratio_1 la misma fracción; ratio_3 = 1 − r1 − r2)

Gravity: no mover (apunta a 1.199 ±0.3%)

Por la sensibilidad, estos cambios sumarían aprox ΔQuality ≈ +0.06–0.07 y ΔABV ≈ ~0.00 (los pequeños +/− de temp/color/ratio_2 casi se cancelan). Es decir, se mantiene el alcohol del estilo y rasca un poquito más de calidad.

Validación

In [94]:
cand = best_full.copy()

# micro-ajustes “seguros”
cand['Temperature'] *= 1.05
cand['Color']       *= 1.05
cand['ratio_2']     *= 1.05
# compensa en ratio_1 para mantener suma=1
cand['ratio_1']     *= 0.95  # o ajusta con la función adjust_ratios
cand['ratio_1'], cand['ratio_2'], cand['ratio_3'] = adjust_ratios(cand['ratio_1'], cand['ratio_2'])

Q_new, A_new = predict_quality_abv_from_row(cand)
print("Quality_pred nueva:", Q_new, " | ABV_pred nuevo:", A_new)

Quality_pred nueva: 10.38818115723511  | ABV_pred nuevo: 5.151183194982944


In [95]:
from datetime import datetime
final = cand.copy()  # 'cand' es tu candidato ajustado
final['ratio_3'] = 1 - float(final['ratio_1']) - float(final['ratio_2'])

cols = ['Fermentation_Time','Temperature','pH_Level','Gravity',
        'Bitterness','Color','Brewhouse_Efficiency','Total_Loss',
        'ratio_1','ratio_2','ratio_3']

sheet = pd.DataFrame([{
    'Recipe_ID': f"Pilsner_MAXQ_{datetime.now():%Y%m%d_%H%M}",
    'Beer_Style': 'Pilsner',
    **{c: round(float(final[c]), 3) for c in cols},
    'ABV_pred': round(float(A_new), 3),        # de tu última predicción
    'Quality_pred': round(float(Q_new), 3)
}])

print(sheet.T)  # vista rápida
sheet.to_csv("Pilsner_brew_sheet_MAX_QUALITY_v2.csv", index=False)

                                               0
Recipe_ID             Pilsner_MAXQ_20250922_1713
Beer_Style                               Pilsner
Fermentation_Time                         12.036
Temperature                                9.434
pH_Level                                   3.989
Gravity                                    1.199
Bitterness                                27.503
Color                                     10.948
Brewhouse_Efficiency                      82.155
Total_Loss                                10.369
ratio_1                                    0.472
ratio_2                                    0.368
ratio_3                                     0.16
ABV_pred                                   5.151
Quality_pred                              10.388


#  Conclusión

Se desarrolló un flujo reproducible para optimizar la calidad de cerveza por estilo con datos reales de producción. El modelo de ABV (RidgeCV con OHE+escalado) logró R² = 0.9976 y RMSE = 0.044 %ABV, suficiente para imponer una restricción confiable en la optimización. Para Quality_Score, se entrenaron modelos por estilo (PolynomialFeatures grado 2 + RidgeCV con validación temporal). Solo Pilsner superó a su baseline (RMSE_test 0.426 vs 0.470), por lo que se procedió a optimizarlo.

Mediante muestreo Monte Carlo (30k candidatos) con receta composicional (Dirichlet) y penalización por salir del rango de ABV del estilo, se obtuvo un TOP-5 de condiciones. Se seleccionó el set-point de máxima calidad y, con un análisis de sensibilidad ±5%, se aplicaron micro-ajustes seguros (↑Temperatura, ↑Color, ↑ratio_2; ↓ratio_1; Gravity estable). El resultado final mejoró la predicción de calidad de 10.326 → 10.388 manteniendo ABV_pred ≈ 5.151% dentro del rango de Pilsner (4.0–5.5%). Esto entrega una hoja de proceso operable (brew sheet) y una metodología clara para repetir en otros estilos.

Implicaciones y recomendaciones

Gravity es la palanca crítica (mueve fuertemente el ABV); debe controlarse con estrecha tolerancia.

Cambios moderados en Temperatura, Color y ratio_2 pueden elevar calidad sin afectar el ABV.

Los demás estilos no superaron su baseline; probablemente requieren mayor flexibilidad del modelo o más variables de proceso.

Limitaciones

La calidad sensorial es ruidosa y estilo-dependiente; faltan variables (levadura/lote, oxígeno, nutrientes, tiempos intermedios).

Los modelos se calibran dentro de rangos históricos; evitar extrapolar fuera de ellos.

Siguientes pasos

Validar en un lote piloto el set-point final y registrar resultados reales (ABV, OG/FG, panel).

Re-entrenar incorporando esos lotes (mejora continua).

Para estilos sin mejora: probar grado 3, ampliar alphas, o añadir la feature abv_delta (desvío al centro del rango).

(Opcional) Introducir penalización por pérdidas para construir un frente calidad–eficiencia.

En síntesis, el proyecto entrega un MVP robusto: modelado fiable de ABV, un modelo útil de calidad para Pilsner y una ruta práctica de optimización que ya produjo un set-point de mayor calidad manteniendo el estilo.